## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [7]:
import mlflow


# mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_tracking_uri("http://127.0.0.1:5001")


ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5001'


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/190105288112393163', creation_time=1761715763762, experiment_id='190105288112393163', last_update_time=1761715763762, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/29 05:31:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/29 05:31:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/190105288112393163/7f885b3d2b3341be8180f964ccdc29aa/artifacts'
🏃 View run bedecked-shrike-643 at: http://127.0.0.1:5001/#/experiments/190105288112393163/runs/7f885b3d2b3341be8180f964ccdc29aa
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/190105288112393163


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/190105288112393163', creation_time=1761715763762, experiment_id='190105288112393163', last_update_time=1761715763762, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

### Interacting with the model registry

In [12]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5001")

In [13]:
client.search_registered_models()

[]

In [14]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

IndexError: list index out of range